In [1]:
%pip install -U transformers datasets peft evaluate tf-keras sacrebleu rouge_score pycocoevalcap nltk

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached datasets-3.2.0-py3-none-any.whl (480 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7
  Attempting uninstall: datasets
    Found existing installation: datasets 2.9.0
    Uninstalling datasets-2.9.0:
      Successfully uninstalled datasets-2.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nlg-metricverse 0.9.9 requires datasets<2.10,>=2.9, but you have datasets 3.2.0 which is incompatible.
nlg-metricverse 0.9.9 requires nltk<3.7.1,>=3.6.6, but you have nltk 3.9.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, DataCollatorForSeq2Seq, AutoTokenizer, get_linear_schedule_with_warmup
from datasets import load_dataset
import evaluate
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from tqdm.auto import tqdm
import re
import nltk
import pandas as pd
from itertools import groupby
from datasets import Dataset
# Update the training and evaluation loop
from torch.utils.data import DataLoader


2025-01-29 11:44:16.788713: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-29 11:44:16.812439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738151056.833152    7421 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738151056.839312    7421 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-29 11:44:16.865822: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
ranks = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
alpha=32
weight_decay = 0.01
dropout_prob = 0.1
label_smooth = 0.1
length_penalty = 0.9
max_length = 128
batch_size=8
lr=2e-4
num_epochs=2
num_warmup_steps=500
train_set_size=None
eval_set_size=None

In [5]:
# https://huggingface.co/docs/transformers/preprocessing
nltk.download("punkt", quiet=True)
model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name) #GPT2Tokenizer.from_pretrained('gpt2-medium')
tokenizer.padding_side = 'left'
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
encoded_input = tokenizer(["This is an example sentence for the rank evaluation task."], truncation=True, padding='max_length', max_length=128)
print(encoded_input)
print(tokenizer.decode(encoded_input["input_ids"][0]))

{'input_ids': [[50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 1212, 318, 281, 1672, 6827, 329, 262, 4279, 12660, 4876, 13]], 'attention_mask': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [6]:
dataset = load_dataset("squad")

train_set = dataset["train"]
test_set = dataset["validation"]
train_set[0]["answers"]["text"][0]

'Saint Bernadette Soubirous'

In [7]:
# Grouping function for E2E NLG test dataset
def preprocess_test_set(test_data):
    return [sample["answers"]["text"][0] for sample in test_data]

def preprocess_train_set(examples):
    questions = examples['question']
    context = [context[:512] for context in examples["context"]]
    answer = [sample["text"][0] for sample in examples['answers']]
    texts = ["<Q>" + q + "<C>" + c + '<A>' + a + " " + tokenizer.eos_token for q, c, a in zip(questions, context, answer)]
    model_inputs = tokenizer(texts, truncation=True)
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

def preprocess_eval(examples):
    questions = examples['question']
    context = [context[:512] for context in examples["context"]]
    answer = [sample["text"][0] for sample in examples['answers']]
    texts = ["<Q>" + q + "<C>" + c + '<A>' for q, c in zip(questions, context)]
    model_inputs = tokenizer(texts, truncation=True)
    model_inputs["question"] = texts
    model_inputs["answer"] = answer
    return model_inputs

In [ ]:
# needs to be installed after dataset has been loaded
%pip install nlg-metricverse

In [9]:
from nlgmetricverse import NLGMetricverse, load_metric

bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')
nist = evaluate.load('nist_mt')
cider = NLGMetricverse(metrics=load_metric("cider"))
meteor = NLGMetricverse(metrics=load_metric("meteor"))

In [10]:
def custom_collate_fn(batch):
    answer = [item['answer'] for item in batch]
    question = [item['question'] for item in batch]
    # Remove 'human_reference' before using data_collator
    batch = [{'input_ids': item['input_ids'], 'attention_mask': item['attention_mask']} for item in batch]
    batch = data_collator(batch)
    batch['answer'] = answer
    batch['question'] = question
    return batch

def generate_predictions(test_dataloader, model, tokenizer, length_penalty):
    model.eval()
    predictions = []
    references = []
    progress_bar = tqdm(test_dataloader, desc="Generating predictions")
    for batch in progress_bar:
        # Use only the meaning representation as input
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        questions = batch['question']
        answers = batch['answer']
        with torch.no_grad():
            output_ids = model.generate(
                input_ids,
                attention_mask=attention_mask,
                num_beams=10,
                length_penalty=length_penalty,
                repetition_penalty=1.0,
                no_repeat_ngram_size=4,
                max_new_tokens=64,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        for i, output in enumerate(output_ids):
            prediction = tokenizer.decode(output, skip_special_tokens=True)
            input_text = questions[i]
            if prediction.startswith(input_text):
                prediction = prediction[len(input_text) :]
            prediction = prediction.strip()
            predictions.append(format_outputs(prediction))
            references.append(format_outputs(answers[i]))
    return predictions, references

def format_outputs(text):
    text = text.lower()
    text = ' '.join(re.split('(\W)', text))
    text = text.split()
    text = ' '.join(text)
    return text

In [11]:
train_size = 20000
test_size = 500
test_data = preprocess_test_set(dataset["validation"])
preprocess_function = preprocess_train_set
preprocess_function_eval = preprocess_eval

train_data = Dataset.from_dict(dataset["train"][:train_size])
test_data = Dataset.from_dict(dataset["validation"][:test_size])

train_tokenized = train_data.map(preprocess_function, batched=True, remove_columns=train_data.column_names)
if train_set_size is not None:
    train_tokenized = train_tokenized.select(range(train_set_size))
test_tokenized = test_data.map(preprocess_function_eval, batched=True, remove_columns=test_data.column_names)
if eval_set_size is not None:
    test_tokenized = test_tokenized.select(range(eval_set_size))
# Data collator and DataLoaders
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, return_tensors="pt", padding=True)

train_dataloader = DataLoader(train_tokenized, shuffle=True, batch_size=batch_size, collate_fn=data_collator)
test_dataloader = DataLoader(test_tokenized, batch_size=1, collate_fn=custom_collate_fn)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:

for rank in ranks:
    peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=rank,
    lora_alpha=alpha,
    target_modules=["c_attn"],
    lora_dropout=dropout_prob,
    init_lora_weights="gaussian",
    bias="none"
    )
    
    model = GPT2LMHeadModel.from_pretrained(model_name)
    # FREEZE WEIGHTS
    #for param in model.parameters():
    #    param.requires_grad = False
    if rank != 0:
        model = get_peft_model(model, peft_config)
    model = model.to(device)
    with torch.no_grad():
        model.resize_token_embeddings(len(tokenizer))
    model.config.pad_token_id = tokenizer.pad_token_id
    print("Evaluation for Rank: " + str(rank))
    if rank != 0:
        print(model.print_trainable_parameters())

    # Optimizer and scheduler
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    num_training_steps = num_epochs * len(train_dataloader)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for batch in progress_bar:
            inputs = {key: val.to(device) for key, val in batch.items()}
            outputs = model(**inputs)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            progress_bar.set_postfix(loss=loss.item())
    
    # Evaluation
    predictions, references = generate_predictions(test_dataloader, model, tokenizer, length_penalty=length_penalty)
    bleu_score = bleu.compute(predictions=predictions, references=references)
    meteor_score = meteor(predictions=predictions, references=references)
    rouge_score = rouge.compute(predictions=predictions, references=references)
    nist_score = nist.compute(predictions=predictions, references=references)
    cider_score = cider(predictions=predictions, references=references)

    test_metrics = {
        'bleu': bleu_score['bleu'],
        'meteor': meteor_score['meteor']['score'],
        'rouge_l': rouge_score['rougeL'],
        'nist': nist_score['nist_mt'],
        'cider': cider_score['cider']['score']
    }

    print(test_metrics)

/opt/conda/lib/python3.11/site-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Evaluation for Rank: 8
trainable params: 786,432 || all params: 355,610,624 || trainable%: 0.2211
None


Epoch 1/2:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch 2/2:   0%|          | 0/2500 [00:00<?, ?it/s]

Generating predictions:   0%|          | 0/500 [00:00<?, ?it/s]

...

Epoch 2/2:   0%|          | 0/2500 [00:00<?, ?it/s]

Generating predictions:   0%|          | 0/500 [00:00<?, ?it/s]

{'bleu': 0.35688349197055386, 'meteor': 0.42455361473262354, 'rouge_l': 0.5175045695045697, 'nist': 4.4107851928493265, 'cider': 0.09167877552176247}
Evaluation for Rank: 256
trainable params: 25,165,824 || all params: 379,990,016 || trainable%: 6.6228
None


Epoch 1/2:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch 2/2:   0%|          | 0/2500 [00:00<?, ?it/s]

In [23]:
# backup in case of OutOfMemory
test_dataloader = DataLoader(test_tokenized, batch_size=1, collate_fn=custom_collate_fn)
predictions, references = generate_predictions(test_dataloader, model, tokenizer, length_penalty=length_penalty)
bleu_score = bleu.compute(predictions=predictions, references=references)
meteor_score = meteor(predictions=predictions, references=references)
rouge_score = rouge.compute(predictions=predictions, references=references)
nist_score = nist.compute(predictions=predictions, references=references)
cider_score = cider(predictions=predictions, references=references)

test_metrics = {
    'bleu': bleu_score['bleu'],
    'meteor': meteor_score['meteor']['score'],
    'rouge_l': rouge_score['rougeL'],
    'nist': nist_score['nist_mt'],
    'cider': cider_score['cider']['score']
}

print(test_metrics)

Generating predictions:   0%|          | 0/500 [00:00<?, ?it/s]

{'bleu': 0.37588958413318463, 'meteor': 0.4170101041124922, 'rouge_l': 0.5083502664002665, 'nist': 4.426698893183452, 'cider': 0.09167877552176247}


In [18]:
def generate_predictions(test_dataloader, model, tokenizer, length_penalty):
    model.eval()
    predictions = []
    references = []
    progress_bar = tqdm(test_dataloader, desc="Generating predictions")
    for idx, batch in enumerate(progress_bar):
        if idx > 3:
            break
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        questions = batch['question']
        answers = batch['answer']
        with torch.no_grad():
            output_ids = model.generate(
                input_ids,
                attention_mask=attention_mask,
                num_beams=10,
                length_penalty=length_penalty,
                repetition_penalty=1.0,
                no_repeat_ngram_size=4,
                max_new_tokens=64,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        for i, output in enumerate(output_ids):
            prediction = tokenizer.decode(output, skip_special_tokens=True)
            input_text = questions[i]
            prediction = prediction.strip()
            predictions.append(format_outputs(prediction))
            references.append(format_outputs(answers[i]))
            print(f"Input Text: {input_text}")
            print(f"Reference Text: {format_outputs(answers[i])}")
            print(f"Prediction: {format_outputs(prediction)}")
            print("\n\n\n")
    return predictions, references

predictions, references = generate_predictions(test_dataloader, model, tokenizer, length_penalty=length_penalty)

Generating predictions:   0%|          | 0/63 [00:00<?, ?it/s]

Input Text: <Q>Which NFL team represented the AFC at Super Bowl 50?<C>Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with<A>
Reference Text: denver broncos
Prediction: denver broncos




Input Text: <Q>Which NFL team represented the NFC at Super Bowl 50?<C>Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn t